## Merging Caracteristiques & Lieux (2019-2023) datasets

In [65]:
import pandas as pd

pd.set_option('display.max_columns', None)

In [ ]:
# Read CSVs into df

details2019 = pd.read_csv('caracteristiques-2019.csv', sep=';')
details2020 = pd.read_csv('caracteristiques-2020.csv', sep=';')
details2021 = pd.read_csv('carcteristiques-2021.csv', sep=';')
details2022 = pd.read_csv('carcteristiques-2022.csv', sep=';')
details2023 = pd.read_csv('caracteristiques-2023.csv', sep=';')
place2019 = pd.read_csv('lieux-2019.csv', sep=';')
place2020 = pd.read_csv('lieux-2020.csv', sep=';')
place2021 = pd.read_csv('lieux-2021.csv', sep=';')
place2022 = pd.read_csv('lieux-2022.csv', sep=';')
place2023 = pd.read_csv('lieux-2023.csv', sep=';')


/var/folders/x9/djcjfydx371712drzs1v_vy80000gn/T/ipykernel_45263/2218030682.py:9: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  place2022 = pd.read_csv('lieux-2022.csv', sep=';')
/var/folders/x9/djcjfydx371712drzs1v_vy80000gn/T/ipykernel_45263/2218030682.py:10: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  place2023 = pd.read_csv('lieux-2023.csv', sep=';')


In [51]:
# Combine details dfs

details_combined = pd.concat(
    [details2019, details2020, details2021, details2022, details2023],
    ignore_index=True
)

print(f'df shape: {details_combined.shape}')

details_combined.info()

df shape: (273226, 16)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273226 entries, 0 to 273225
Data columns (total 16 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Num_Acc      217924 non-null  float64
 1   jour         273226 non-null  int64  
 2   mois         273226 non-null  int64  
 3   an           273226 non-null  int64  
 4   hrmn         273226 non-null  object 
 5   lum          273226 non-null  int64  
 6   dep          273226 non-null  object 
 7   com          273226 non-null  object 
 8   agg          273226 non-null  int64  
 9   int          273226 non-null  int64  
 10  atm          273226 non-null  int64  
 11  col          273226 non-null  int64  
 12  adr          269247 non-null  object 
 13  lat          273226 non-null  object 
 14  long         273226 non-null  object 
 15  Accident_Id  55302 non-null   float64
dtypes: float64(2), int64(8), object(6)
memory usage: 33.4+ MB


In [49]:
# Combine place dfs

place_combined = pd.concat(
    [place2019, place2020, place2021, place2022, place2023],
    ignore_index=True
)

print(f'df shape: {place_combined.shape}')

place_combined.info()

df shape: (289264, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289264 entries, 0 to 289263
Data columns (total 18 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Num_Acc  289264 non-null  int64  
 1   catr     289264 non-null  int64  
 2   voie     253443 non-null  object 
 3   v1       278518 non-null  float64
 4   v2       23160 non-null   object 
 5   circ     289264 non-null  int64  
 6   nbv      289264 non-null  object 
 7   vosp     289264 non-null  int64  
 8   prof     289264 non-null  int64  
 9   pr       289264 non-null  object 
 10  pr1      289264 non-null  object 
 11  plan     289264 non-null  int64  
 12  lartpc   527 non-null     object 
 13  larrout  230796 non-null  object 
 14  surf     289264 non-null  int64  
 15  infra    289264 non-null  int64  
 16  situ     289264 non-null  int64  
 17  vma      289264 non-null  int64  
dtypes: float64(1), int64(10), object(7)
memory usage: 39.7+ MB


In [52]:
# Inner join details and place df on 'Num_Acc'
df = pd.merge(
    details_combined,      # left DataFrame
    place_combined,        # right DataFrame
    on='Num_Acc',          # column to join on
    how='inner'            # inner join
)

In [56]:
df.shape

(233962, 33)

In [62]:
# Rename columns

df.rename(columns={
    'Num_Acc': 'accident_number',
    'jour': 'day',
    'mois': 'month',
    'an': 'year',
    'hrmn': 'hour_minute',
    'lum': 'light_conditions',
    'dep': 'department',
    'com': 'commune',
    'agg': 'urban_area',
    'int': 'intersection_type',
    'atm': 'weather',
    'col': 'collision_type',
    'adr': 'road_address',
    'lat': 'latitude',
    'long': 'longitude',
    'Accident_Id': 'accident_uid',
    'catr': 'road_category',
    'voie': 'lane_type',
    'v1': 'vehicle_1_type',
    'v2': 'vehicle_2_type',
    'circ': 'traffic_condition',
    'nbv': 'num_vehicles',
    'vosp': 'special_vehicle',
    'prof': 'road_profile',
    'pr': 'road_ref_1',
    'pr1': 'road_ref_2',
    'plan': 'road_layout',
    'lartpc': 'road_alignment',
    'larrout': 'road_side',
    'surf': 'road_surface',
    'infra': 'road_infra',
    'situ': 'accident_situation',
    'vma': 'speed_limit'
}, inplace=True)

In [63]:
# Drop accident_number and accident_uid columns

df = df.drop(columns=['accident_number', 'accident_uid'])

In [64]:
# Combine day, month, year columns
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])

# Create day of the week column
df['day_of_week'] = df['date'].dt.day_name()

# Drop old columns
df.drop(columns=['year', 'month', 'day'], inplace=True)

# Move new columns to the front
cols = ['date', 'day_of_week'] + [c for c in df.columns if c not in ['date', 'day_of_week']]
df = df[cols]

In [66]:
df.head()

,date,day_of_week,hour_minute,light_conditions,department,commune,urban_area,intersection_type,weather,collision_type,road_address,latitude,longitude,road_category,lane_type,vehicle_1_type,vehicle_2_type,traffic_condition,num_vehicles,special_vehicle,road_profile,road_ref_1,road_ref_2,road_layout,road_alignment,road_side,road_surface,road_infra,accident_situation,speed_limit
0,2019-11-30,Saturday,01:30,4,93,93053,1,1,1,2,AUTOROUTE A3,"48,8962100","2,4701200",1,3,0.0,NaN,3,10,0,1,6,900,2,NaN,NaN,1,2,1,70
1,2019-11-30,Saturday,02:50,3,93,93066,1,1,1,6,AUTOROUTE A1,"48,9307000","2,3688000",1,1,0.0,NaN,1,2,0,4,3,845,2,NaN,NaN,1,0,1,70
2,2019-11-28,Thursday,15:15,1,92,92036,1,1,1,4,AUTOROUTE A86,"48,9358718","2,3191744",1,86,0.0,NaN,3,8,0,1,10,500,3,NaN,NaN,1,0,1,90
3,2019-11-30,Saturday,20:20,5,94,94069,1,1,1,4,A4,"48,8173295","2,4281502",1,4,0.0,NaN,3,5,0,1,2,299,1,NaN,NaN,1,0,1,90
4,2019-11-30,Saturday,04:00,3,94,94028,1,1,1,2,A86 INT,"48,7763620","2,4332540",1,86,0.0,INT,1,3,0,1,41,0,3,NaN,NaN,1,2,1,90


In [71]:
# Null values

# Absolute number of missing values per column
null_counts = df.isnull().sum()

# Percentage of missing values per column
null_percentage = df.isnull().mean() * 100

# Combine null stats
missing_summary = pd.DataFrame({
    'null_count': null_counts,
    'null_percentage': null_percentage
}).sort_values(by='null_count', ascending=False)

missing_summary[missing_summary['null_percentage']>0]

,null_count,null_percentage
road_alignment,233463,99.786718
vehicle_2_type,216057,92.347048
road_side,58468,24.990383
lane_type,31016,13.256854
vehicle_1_type,10746,4.593054
road_address,2997,1.280977


In [73]:
# Export df to CSV
df.to_csv('df.csv', index=False)